In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
drone_path_1 = '../../data/1m/*.wav'
drone_path_10 = '../../data/10m/*.wav'
drone_path_20 = '../../data/20m/*.wav'
drone_path_30 = '../../data/30m/*.wav'
drone_path_40 = '../../data/40m/*.wav'
drone_path_50 = '../../data/50m/*.wav'
background_path = '../../data/background/*.wav'

drone_files_1 = glob.glob(drone_path_1)
drone_files_10 = glob.glob(drone_path_10)
drone_files_20 = glob.glob(drone_path_20)
drone_files_30 = glob.glob(drone_path_30)
drone_files_40 = glob.glob(drone_path_40)
drone_files_50 = glob.glob(drone_path_50)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
drone_raw_1 = load(drone_files_1)
drone_raw_10 = load(drone_files_10)
drone_raw_20 = load(drone_files_20)
drone_raw_30 = load(drone_files_30)
drone_raw_40 = load(drone_files_40)
drone_raw_50 = load(drone_files_50)

background_raw = load(background_files)

(2088960,)
(2088960,)
(2088960,)
(2072565,)
(2085722,)
(2088960,)
(6032517,)


# Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_drone_1, y_drone_1 = mfcc4(drone_raw_1, 0)
mfcc_drone_10, y_drone_10 = mfcc4(drone_raw_10, 1)
mfcc_drone_20, y_drone_20 = mfcc4(drone_raw_20, 2)
mfcc_drone_30, y_drone_30 = mfcc4(drone_raw_30, 3)
mfcc_drone_40, y_drone_40 = mfcc4(drone_raw_40, 4)
mfcc_drone_50, y_drone_50 = mfcc4(drone_raw_50, 5)

mfcc_background, y_background = mfcc4(background_raw, 6)

(2088960,)
small end: (16, 9)
(2088960,)
small end: (16, 9)
(2088960,)
small end: (16, 9)
(2072565,)
small end: (16, 16)
small end: (16, 8)
(2085722,)
small end: (16, 10)
small end: (16, 2)
(2088960,)
small end: (16, 9)
(6032517,)
small end: (16, 15)
small end: (16, 7)


In [8]:
print(mfcc_drone_1.shape, y_drone_1.shape)
print(mfcc_drone_10.shape, y_drone_10.shape)
print(mfcc_drone_20.shape, y_drone_20.shape)
print(mfcc_drone_30.shape, y_drone_30.shape)
print(mfcc_drone_40.shape, y_drone_40.shape)
print(mfcc_drone_50.shape, y_drone_50.shape)
print(mfcc_background.shape, y_background.shape)

(509, 16, 16) (509,)
(509, 16, 16) (509,)
(509, 16, 16) (509,)
(504, 16, 16) (504,)
(508, 16, 16) (508,)
(509, 16, 16) (509,)
(1471, 16, 16) (1471,)


In [9]:
X = np.concatenate((mfcc_drone_1,mfcc_drone_10,mfcc_drone_20,mfcc_drone_30,mfcc_drone_40,mfcc_drone_50, mfcc_background), axis=0)
y = np.hstack((y_drone_1, y_drone_10, y_drone_20, y_drone_30, y_drone_40, y_drone_50, y_background))
print(X.shape, y.shape)

(4519, 16, 16) (4519,)


In [10]:
X = np.reshape(X,(X.shape[0],-1))

In [11]:
X.shape

(4519, 256)

In [12]:
n_labels = y.shape[0]
n_unique_labels = 7
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(4519, 7)


In [13]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [14]:
print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(2892, 256) (2892, 7)
(723, 256) (723, 7)
(904, 256) (904, 7)


In [15]:
# Save Data
np.save('../../model/X_train', X_train)
np.save('../../model/X_test', X_test)
np.save('../../model/X_val', X_val)
np.save('../../model/y_val', y_val)
np.save('../../model/y_train', y_train)
np.save('../../model/y_test', y_test)

### Until this part

In [16]:
# Load Data
X_train = np.load('../../model/X_train.npy')
X_test = np.load('../../model/X_test.npy')
X_val = np.load('../../model/X_val.npy')
y_val = np.load('../../model/y_val.npy')
y_train = np.load('../../model/y_train.npy')
y_test = np.load('../../model/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout


##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [17]:
tf.reset_default_graph()

In [18]:
n_mfcc = 16
n_frame = 16
n_classes = 7
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.0002  # 0.005
training_epochs = 500 # 수정해야해

# Layer

## Experiment
- learning rate 0.0002
- no cnn/ pooling 
- just dense with dropout 

In [19]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

keep_prob = tf.placeholder(tf.float32)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)
dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)
dense3 = tf.layers.dense(inputs=dropout2, units=512, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)
dense4 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
dropout4 = tf.nn.dropout(dense4, keep_prob=keep_prob)
dense5 = tf.layers.dense(inputs=dropout4, units=256, activation=tf.nn.relu)
dropout5 = tf.nn.dropout(dense5, keep_prob=keep_prob)

logits= tf.layers.dense(inputs=dropout5, units=7)

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [37]:
#sess = tf.Session()
#
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())

In [38]:
# model save
model_path = '../../model/DNN/7_dense_model'
saver = tf.train.Saver()

# Trainning

In [39]:
import math

In [40]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
#original 
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)
with tf.device("/gpu:0"):
    for epoch in range(1,300):
        avg_cost = 0    
#    total_batch = int(X_train.shape[0] / batch_size)
        for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
            x_ = X_train[batch_size * i: batch_size * (i + 1)]
            y_ = y_train[batch_size * i: batch_size * (i + 1)]
            feed_dict={X:x_, Y:y_, keep_prob:0.6}    
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            cost_history = np.append(cost_history,cost)
            avg_cost += c
            #accuracy_val = sess.run([accuracy], feed_dict={X:X_val, Y:y_val})
        val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_val , keep_prob:1})  
        val_true = sess.run(tf.argmax(y_val,1))
        accuracy_val = accuracy_score(val_pred, val_true)
        print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),)
saver.save(sess, model_path)

Epoch: 0001 cost =  1.260486548 	검증 세트 정확도: 30.290%
Epoch: 0002 cost =  0.356774255 	검증 세트 정확도: 28.077%
Epoch: 0003 cost =  0.169232626 	검증 세트 정확도: 38.036%
Epoch: 0004 cost =  0.102083877 	검증 세트 정확도: 43.845%
Epoch: 0005 cost =  0.074644716 	검증 세트 정확도: 44.952%
Epoch: 0006 cost =  0.061628814 	검증 세트 정확도: 46.611%
Epoch: 0007 cost =  0.055182774 	검증 세트 정확도: 45.367%
Epoch: 0008 cost =  0.050316864 	검증 세트 정확도: 51.037%
Epoch: 0009 cost =  0.045726227 	검증 세트 정확도: 57.123%
Epoch: 0010 cost =  0.043110019 	검증 세트 정확도: 58.230%
Epoch: 0011 cost =  0.040265841 	검증 세트 정확도: 64.869%
Epoch: 0012 cost =  0.039394458 	검증 세트 정확도: 67.082%
Epoch: 0013 cost =  0.037533719 	검증 세트 정확도: 62.241%
Epoch: 0014 cost =  0.037190467 	검증 세트 정확도: 61.549%
Epoch: 0015 cost =  0.035854630 	검증 세트 정확도: 61.964%
Epoch: 0016 cost =  0.035316632 	검증 세트 정확도: 60.858%
Epoch: 0017 cost =  0.034552740 	검증 세트 정확도: 61.549%
Epoch: 0018 cost =  0.033565900 	검증 세트 정확도: 62.517%
Epoch: 0019 cost =  0.032663211 	검증 세트 정확도: 65.145%
Epoch: 0020 

Epoch: 0159 cost =  0.008826407 	검증 세트 정확도: 94.467%
Epoch: 0160 cost =  0.009035328 	검증 세트 정확도: 87.414%
Epoch: 0161 cost =  0.008722789 	검증 세트 정확도: 90.456%
Epoch: 0162 cost =  0.008169196 	검증 세트 정확도: 93.638%
Epoch: 0163 cost =  0.007340604 	검증 세트 정확도: 95.297%
Epoch: 0164 cost =  0.007896530 	검증 세트 정확도: 94.329%
Epoch: 0165 cost =  0.007965227 	검증 세트 정확도: 93.638%
Epoch: 0166 cost =  0.008182946 	검증 세트 정확도: 94.329%
Epoch: 0167 cost =  0.006892680 	검증 세트 정확도: 90.180%
Epoch: 0168 cost =  0.006801100 	검증 세트 정확도: 94.467%
Epoch: 0169 cost =  0.006777055 	검증 세트 정확도: 90.871%
Epoch: 0170 cost =  0.006875784 	검증 세트 정확도: 93.084%
Epoch: 0171 cost =  0.006934608 	검증 세트 정확도: 94.606%
Epoch: 0172 cost =  0.006710479 	검증 세트 정확도: 91.563%
Epoch: 0173 cost =  0.008095129 	검증 세트 정확도: 94.744%
Epoch: 0174 cost =  0.007233485 	검증 세트 정확도: 92.393%
Epoch: 0175 cost =  0.006844003 	검증 세트 정확도: 92.946%
Epoch: 0176 cost =  0.006288816 	검증 세트 정확도: 94.744%
Epoch: 0177 cost =  0.006734947 	검증 세트 정확도: 94.053%
Epoch: 0178 

'../../model/DNN/7_dense_model'

## Prediction

In [43]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) 
y_true = sess.run(tf.argmax(y_test,1))

In [44]:
# Ptint Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.948
Accuracy:  0.9480088495575221
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       116
           1       0.99      0.89      0.94        90
           2       0.73      0.95      0.83       109
           3       0.99      0.99      0.99        93
           4       0.99      1.00      1.00       101
           5       0.99      0.72      0.84        98
           6       0.98      1.00      0.99       297

   micro avg       0.95      0.95      0.95       904
   macro avg       0.95      0.93      0.94       904
weighted avg       0.96      0.95      0.95       904

[[113   0   0   1   0   0   2]
 [  0  80  10   0   0   0   0]
 [  0   1 104   0   0   1   3]
 [  0   0   0  92   1   0   0]
 [  0   0   0   0 101   0   0]
 [  0   0  27   0   0  71   0]
 [  0   0   1   0   0   0 296]]
